## Generate 3D features

### Using RDKit/MORDRED

**Created on 19th May, 2022; remodified on 13th February, 2024**

In [1]:
import warnings
warnings.filterwarnings('ignore')
#from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
#import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, LassoCV, \
    RidgeClassifier, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR, SVC
from sklearn.gaussian_process import GaussianProcessRegressor, GaussianProcessClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel, WhiteKernel, \
RationalQuadratic, ExpSineSquared, DotProduct
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, \
precision_score, auc
import sys, os, csv
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import pickle
from rdkit import Chem
from ifg import identify_functional_groups ## Need ifg.py file
import rdkit.Chem.rdMolDescriptors as MolDescriptors
import rdkit.Chem.Descriptors as Descriptors
import rdkit.Chem.Lipinski as Lipinski
from rdkit.Chem import AllChem, DataStructs, Fragments, Descriptors3D

In [23]:
%%bash
pwd
ls -ltr

/Users/riteshk/Library/CloudStorage/Box-Box/Research-postdoc/liquid-electrolyte-ML/Science-rev-1/ionic-conductivity_2.1/dataset_v2.0/common-log-output/lgbm_trials/generate-qsar-feat
total 10264
-rw-------@ 1 riteshk  staff     1618 Feb 13 20:46 list_unique_solv-salt-add.csv
-rw-------@ 1 riteshk  staff     4144 Feb 13 20:49 ifg.py
drwxr-xr-x  3 riteshk  staff       96 Feb 13 20:50 __pycache__
-rw-------@ 1 riteshk  staff   787294 Feb 13 21:50 all_multi_comp_comm.csv
-rw-------@ 1 riteshk  staff   765127 Feb 13 21:50 all_multi_comp_comb_comm.csv
-rw-------@ 1 riteshk  staff      405 Feb 13 21:54 solvent_functional_group_identifier.csv
-rw-------@ 1 riteshk  staff      591 Feb 13 21:54 salt_functional_group_identifier.csv
-rw-------@ 1 riteshk  staff   626696 Feb 13 21:57 all_multi_comp_add.csv
-rw-------@ 1 riteshk  staff  3045402 Feb 13 22:07 generate_qsar_functional_grp_cond.ipynb


In [3]:
df = pd.read_csv('all_multi_comp_comm.csv')
df

,solv_1_sm,solv_2_sm,solv_3_sm,solv_4_sm,salt_sm,conductivity_log
0,O=C1OCCO1,COC(=O)OC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,1.068186
1,O=C1OCCO1,CCCCOC(=O)OC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,0.770852
2,O=C1OCCO1,CC(C)COC(=O)OC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,0.732394
3,O=C1OCCO1,CCC(C)OC(=O)OC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,0.812913
4,CCOC(=O)OCC,NaN,NaN,NaN,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.322219
...,...,...,...,...,...,...
10191,CS(=O)C,COCCOC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,0.672098
10192,CS(=O)C,COCCOC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,1.139879
10193,CS(=O)C,COCCOC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,1.204120
10194,CS(=O)C,COCCOC,NaN,NaN,[Li+].F[P-](F)(F)(F)(F)F,1.167317


In [36]:
# df_comb['solv_comb_sm'] = df.apply(lambda row: '.'.join([str(x) for x in row[['solv_1_sm', 'solv_2_sm', 'solv_3_sm', 'solv_4_sm']] if pd.notnull(x)]), axis=1)
# df_comb

,solv_comb_sm,salt_sm,conductivity_log
0,O=C1OCCO1.COC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,1.068186
1,O=C1OCCO1.CCCCOC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,0.770852
2,O=C1OCCO1.CC(C)COC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,0.732394
3,O=C1OCCO1.CCC(C)OC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,0.812913
4,CCOC(=O)OCC,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.322219
...,...,...,...
10191,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,0.672098
10192,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,1.139879
10193,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,1.204120
10194,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,1.167317


In [41]:
# df_comb.to_csv('all_multi_comp_comb_comm.csv', index=False)

In [24]:
df_comb = pd.read_csv('all_multi_comp_comb_comm.csv')
df_add = pd.read_csv('all_multi_comp_add.csv')
df_comb

,solv_comb_sm,salt_sm,conductivity_log
0,O=C1OCCO1.COC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,1.068186
1,O=C1OCCO1.CCCCOC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,0.770852
2,O=C1OCCO1.CC(C)COC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,0.732394
3,O=C1OCCO1.CCC(C)OC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,0.812913
4,CCOC(=O)OCC,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.322219
...,...,...,...
10191,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,0.672098
10192,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,1.139879
10193,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,1.204120
10194,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,1.167317


In [9]:
unique_solv = list(set(df['solv_1_sm'].unique().tolist() + df['solv_2_sm'].unique().tolist() + df['solv_3_sm'].unique().tolist() + df['solv_4_sm'].unique().tolist()))
unique_salt = list(set(df['salt_sm'].unique().tolist()))
unique_solv.remove(np.nan)
unique_salt

['[B-](F)(F)(F)F.C1CC[N+]2(C1)CCCC2',
 '[B-](C1=CC=CC=C1)(C2=CC=CC=C2)(C3=CC=CC=C3)C4=CC=CC=C4.CCCC[N+](CCCC)(CCCC)CCCC',
 'CCN1C=C[N+](=C1)C.[B-]12(OC(=O)C(=O)O1)OC(=O)C(=O)O2',
 'C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F.C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F.[Ca+2]',
 'C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F.[Cu+]',
 '[B-](F)(F)(F)F.CCC[N+](CCC)(CCC)CCC',
 'C[N+](C)(C)C.C(F)(F)(F)S(=O)(=O)[O-]',
 'CCN1C=C[N+](=C1)C.O=C1O[B-](F)(F)OC1=O',
 '[Li+].FC(F)(F)S(=O)(=O)[C-](S(=O)(=O)C(F)(F)F)S(=O)(=O)C(F)(F)F',
 'C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F.C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F.C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F.[Y+3]',
 '[Li+].c1ccc2O[B-]3(Oc2c1)Oc1ccccc1O3',
 '[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F',
 'N#Cc1n[n-]nc1C#N.[Li+]',
 '[B-](F)(F)(F)F.CCN1C=C[N+](=C1)C',
 'CCC[N+](CCC)(CCC)CCC.C(F)(F)(F)S(=O)(=O)[O-]',
 'CC[N+](CC)(CC)CC.[O-]Cl(=O)(=O)=O',
 'CCCC[N+](CCCC)(CCCC)CCCC.F[P-](F)(F)(F)(F)F',
 'C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F.C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)

#### Functional group features

Run it only for the first time

In [18]:
all_functional_groups_solv = []
for i in range(len(unique_solv)):
    smile = unique_solv[i]
    print(smile)
    mol = Chem.MolFromSmiles(smile)
    functional_group = identify_functional_groups(mol)
    for j in range(len(functional_group)):
        all_functional_groups_solv.append(functional_group[j].atoms)
unique_functional_groups_solv = list(set(all_functional_groups_solv))
unique_functional_groups_solv

O=C1OCCO1
COC(=O)OC
CCCCOC(=O)OC
CC(C)COC(=O)OC
CCC(C)OC(=O)OC
CCOC(=O)OCC
CCOC(=O)OC(C)F
CC(OC(=O)OCC(F)(F)F)F
FC1COC(=O)O1
C1(C(OC(=O)O1)F)F
CCOC(=O)OC
FC(F)C(F)(F)COC(F)(F)C(F)F
C1C(OC(=O)O1)CC(C(F)(F)F)(C(F)(F)F)F
COC(=O)OCC(F)(F)F
COC(F)(C(F)(F)C(F)(F)F)C(F)(C(F)(F)F)C(F)(F)F
CC(OC(F)(F)C(F)C(F)(F)F)C(F)(F)C(F)C(F)(F)F
CC1COC(=O)O1
CCOCCOCC
ClCCl
C1CCOC1
Cc1ccccc1
CC1CCCO1
COCCOC
C1COCO1
O=c1occo1
C=CCCCOCC1COC(=O)O1
CO[Si](CCCOCC1COC(=O)O1)(OC)OC
CCO[Si](CCCOCC1COC(=O)O1)(OCC)OCC
C[Si](C)(C)O[Si](C)(C)CCCOCC1COC(=O)O1
CCCC(=O)OC
CN1CCCC1=O
CCCCOCCOCCCC
COP(=O)(OC)OC
O=C1CCCO1
COC(=O)OCCCF
CCCOC(=O)OC
COC(=O)OCCC(F)(F)F
COC(=O)OCC(F)(F)C(F)F
COC(=O)OCC(F)(F)C(F)(F)F
O=C1OCC(CF)O1
O=C(OCF)OCF
COC(=O)OCF
COC(=O)OCCF
CCOC(=O)OCCF
CCOC(=O)OCC(F)F
CCOC(=O)OCC(F)(F)F
C[Si](C)(C)O[Si](C)(CCC1COC(=O)O1)O[Si](C)(C)C
C[Si](C)(C)O[Si](C)(C)CCC1COC(=O)O1
CCC1COC(=O)O1
CS(=O)C
C1CCOCC1
CC1CCC(O1)C
CCOC(C(C(C(F)(F)F)(F)F)(F)F)(F)F
COP(=O)(C)OC
C1C(OC(=O)O1)F
FC(F)(F)C(F)(F)COC(F)(F)C(F)(F)F
CC#

['S',
 'o',
 'O=[N+][O-]',
 'OB(O)O',
 'Cl',
 'F[Si]',
 'nnn',
 'NC(N)=S',
 'C=O',
 'O',
 'O=S=O',
 'NC=O',
 'O[Si]',
 'C=C',
 'NS(=O)=O',
 '[Si]O[Si]',
 'NC(N)=O',
 'O=S(O)O',
 'O=C(O)O',
 'O[Si](O)O',
 'C=CS(=O)=O',
 'F[Si]F',
 'O=CC(=O)O',
 '[Si]O[Si]O[Si]',
 'O=P(O)O',
 'OB(O)B(O)O',
 'O=S',
 'OBO',
 'O[Si]O',
 'O=S(=O)F',
 'OCO',
 'OC(O)O',
 'O=S(=O)[N-]S(=O)=O',
 'F',
 '[Si]',
 'C#N',
 'O=CO',
 'NC(O)=S',
 'C=CC#N',
 'n',
 '[N+]',
 'O=P(O)(O)O']

In [19]:
## Assign identifier to each type of functional group in solvent --> changing everytime I run
dict_identifier_solv = {}
for i in range(len(unique_functional_groups_solv)):
    dict_identifier_solv[unique_functional_groups_solv[i]] = i+1
dict_identifier_solv

{'S': 1,
 'o': 2,
 'O=[N+][O-]': 3,
 'OB(O)O': 4,
 'Cl': 5,
 'F[Si]': 6,
 'nnn': 7,
 'NC(N)=S': 8,
 'C=O': 9,
 'O': 10,
 'O=S=O': 11,
 'NC=O': 12,
 'O[Si]': 13,
 'C=C': 14,
 'NS(=O)=O': 15,
 '[Si]O[Si]': 16,
 'NC(N)=O': 17,
 'O=S(O)O': 18,
 'O=C(O)O': 19,
 'O[Si](O)O': 20,
 'C=CS(=O)=O': 21,
 'F[Si]F': 22,
 'O=CC(=O)O': 23,
 '[Si]O[Si]O[Si]': 24,
 'O=P(O)O': 25,
 'OB(O)B(O)O': 26,
 'O=S': 27,
 'OBO': 28,
 'O[Si]O': 29,
 'O=S(=O)F': 30,
 'OCO': 31,
 'OC(O)O': 32,
 'O=S(=O)[N-]S(=O)=O': 33,
 'F': 34,
 '[Si]': 35,
 'C#N': 36,
 'O=CO': 37,
 'NC(O)=S': 38,
 'C=CC#N': 39,
 'n': 40,
 '[N+]': 41,
 'O=P(O)(O)O': 42}

In [20]:
## Save the above identifier
df_identifier_solv = pd.DataFrame([dict_identifier_solv])
df_identifier_solv.to_csv('solvent_functional_group_identifier.csv', index=False)
df_identifier_solv.to_json('solvent_functional_group_identifier.json')
df_identifier_solv

,S,o,O=[N+][O-],OB(O)O,Cl,F[Si],nnn,NC(N)=S,C=O,O,...,O=S(=O)[N-]S(=O)=O,F,[Si],C#N,O=CO,NC(O)=S,C=CC#N,n,[N+],O=P(O)(O)O
0,1,2,3,4,5,6,7,8,9,10,...,33,34,35,36,37,38,39,40,41,42


In [21]:
all_functional_groups_salt = []
for i in range(len(unique_salt)):
    smile = unique_salt[i]
    mol = Chem.MolFromSmiles(smile)
    functional_group = identify_functional_groups(mol)
    for j in range(len(functional_group)):
        all_functional_groups_salt.append(functional_group[j].atoms)
unique_functional_groups_salt = list(set(all_functional_groups_salt))
unique_functional_groups_salt

['[Ba+2]',
 '[Li+]',
 'O=C[O-]',
 '[NH4+]',
 '[K+]',
 'O',
 'O=S=O',
 'O=S(=O)[O-]',
 'n[n-]n',
 '[Br-]',
 'F[Sb-](F)(F)(F)(F)F',
 'O=C1O[B-]2(OC1=O)OC(=O)C(=O)O2',
 'O[B-](O)(O)O',
 'F[As-](F)(F)(F)(F)F',
 'O[P-](O)(O)(O)(O)O',
 'O=[N+]([O-])[O-]',
 'F[B-](F)F',
 'F[P-](F)(F)(F)(F)F',
 'F[B-](F)(F)F',
 'O=S(=O)NS(=O)=O',
 '[Na+]',
 'N',
 '[O-][Cl+3]([O-])([O-])[O-]',
 '[Y+3]',
 'O=S(=O)[N-]S(=O)(=O)F',
 '[La+3]',
 '[Cl-]',
 '[n+]',
 'N#C[N-]C#N',
 'O=S(=O)[N-]S(=O)=O',
 '[Cu+]',
 'O=S(=O)(F)[N-]S(=O)(=O)F',
 'O=C1O[B-](F)(F)OC1=O',
 'F',
 'N#C[S-]',
 '[Ca+2]',
 'O=CO[B-](O)(O)OC=O',
 'C#N',
 'n',
 '[Pb+2]',
 '[n-]',
 '[B-]',
 '[N+]']

In [22]:
## Assign identifier to each type of functional group in salt
dict_identifier_salt = {}
for i in range(len(unique_functional_groups_salt)):
    dict_identifier_salt[unique_functional_groups_salt[i]] = i+1
dict_identifier_salt

{'[Ba+2]': 1,
 '[Li+]': 2,
 'O=C[O-]': 3,
 '[NH4+]': 4,
 '[K+]': 5,
 'O': 6,
 'O=S=O': 7,
 'O=S(=O)[O-]': 8,
 'n[n-]n': 9,
 '[Br-]': 10,
 'F[Sb-](F)(F)(F)(F)F': 11,
 'O=C1O[B-]2(OC1=O)OC(=O)C(=O)O2': 12,
 'O[B-](O)(O)O': 13,
 'F[As-](F)(F)(F)(F)F': 14,
 'O[P-](O)(O)(O)(O)O': 15,
 'O=[N+]([O-])[O-]': 16,
 'F[B-](F)F': 17,
 'F[P-](F)(F)(F)(F)F': 18,
 'F[B-](F)(F)F': 19,
 'O=S(=O)NS(=O)=O': 20,
 '[Na+]': 21,
 'N': 22,
 '[O-][Cl+3]([O-])([O-])[O-]': 23,
 '[Y+3]': 24,
 'O=S(=O)[N-]S(=O)(=O)F': 25,
 '[La+3]': 26,
 '[Cl-]': 27,
 '[n+]': 28,
 'N#C[N-]C#N': 29,
 'O=S(=O)[N-]S(=O)=O': 30,
 '[Cu+]': 31,
 'O=S(=O)(F)[N-]S(=O)(=O)F': 32,
 'O=C1O[B-](F)(F)OC1=O': 33,
 'F': 34,
 'N#C[S-]': 35,
 '[Ca+2]': 36,
 'O=CO[B-](O)(O)OC=O': 37,
 'C#N': 38,
 'n': 39,
 '[Pb+2]': 40,
 '[n-]': 41,
 '[B-]': 42,
 '[N+]': 43}

In [23]:
## Save the above identifier
df_identifier_salt = pd.DataFrame([dict_identifier_salt])
df_identifier_salt.to_csv('salt_functional_group_identifier.csv', index=False)
df_identifier_salt.to_json('salt_functional_group_identifier.json')
df_identifier_salt

,[Ba+2],[Li+],O=C[O-],[NH4+],[K+],O,O=S=O,O=S(=O)[O-],n[n-]n,[Br-],...,F,N#C[S-],[Ca+2],O=CO[B-](O)(O)OC=O,C#N,n,[Pb+2],[n-],[B-],[N+]
0,1,2,3,4,5,6,7,8,9,10,...,34,35,36,37,38,39,40,41,42,43


...till here

Calculate functional group features for solvent

In [11]:
df_solv = pd.read_csv('solvent_functional_group_identifier.csv')
df_solv

,S,o,O=[N+][O-],OB(O)O,Cl,F[Si],nnn,NC(N)=S,C=O,O,...,O=S(=O)[N-]S(=O)=O,F,[Si],C#N,O=CO,NC(O)=S,C=CC#N,n,[N+],O=P(O)(O)O
0,1,2,3,4,5,6,7,8,9,10,...,33,34,35,36,37,38,39,40,41,42


In [17]:
unique_functional_groups_solv = list(df_solv.columns)
# unique_functional_groups_solv
## Assign identifier to each type of functional group in solvent 
dict_identifier_solv = {}
for i in range(len(unique_functional_groups_solv)):
    dict_identifier_solv[unique_functional_groups_solv[i]] = i+1
dict_identifier_solv

{'S': 1,
 'o': 2,
 'O=[N+][O-]': 3,
 'OB(O)O': 4,
 'Cl': 5,
 'F[Si]': 6,
 'nnn': 7,
 'NC(N)=S': 8,
 'C=O': 9,
 'O': 10,
 'O=S=O': 11,
 'NC=O': 12,
 'O[Si]': 13,
 'C=C': 14,
 'NS(=O)=O': 15,
 '[Si]O[Si]': 16,
 'NC(N)=O': 17,
 'O=S(O)O': 18,
 'O=C(O)O': 19,
 'O[Si](O)O': 20,
 'C=CS(=O)=O': 21,
 'F[Si]F': 22,
 'O=CC(=O)O': 23,
 '[Si]O[Si]O[Si]': 24,
 'O=P(O)O': 25,
 'OB(O)B(O)O': 26,
 'O=S': 27,
 'OBO': 28,
 'O[Si]O': 29,
 'O=S(=O)F': 30,
 'OCO': 31,
 'OC(O)O': 32,
 'O=S(=O)[N-]S(=O)=O': 33,
 'F': 34,
 '[Si]': 35,
 'C#N': 36,
 'O=CO': 37,
 'NC(O)=S': 38,
 'C=CC#N': 39,
 'n': 40,
 '[N+]': 41,
 'O=P(O)(O)O': 42}

In [12]:
df_salt = pd.read_csv('salt_functional_group_identifier.csv')
df_salt

,[Ba+2],[Li+],O=C[O-],[NH4+],[K+],O,O=S=O,O=S(=O)[O-],n[n-]n,[Br-],...,F,N#C[S-],[Ca+2],O=CO[B-](O)(O)OC=O,C#N,n,[Pb+2],[n-],[B-],[N+]
0,1,2,3,4,5,6,7,8,9,10,...,34,35,36,37,38,39,40,41,42,43


In [18]:
unique_functional_groups_salt = list(df_salt.columns)
# unique_functional_groups_salt
## Assign identifier to each type of functional group in salt
dict_identifier_salt = {}
for i in range(len(unique_functional_groups_salt)):
    dict_identifier_salt[unique_functional_groups_salt[i]] = i+1
dict_identifier_salt

{'[Ba+2]': 1,
 '[Li+]': 2,
 'O=C[O-]': 3,
 '[NH4+]': 4,
 '[K+]': 5,
 'O': 6,
 'O=S=O': 7,
 'O=S(=O)[O-]': 8,
 'n[n-]n': 9,
 '[Br-]': 10,
 'F[Sb-](F)(F)(F)(F)F': 11,
 'O=C1O[B-]2(OC1=O)OC(=O)C(=O)O2': 12,
 'O[B-](O)(O)O': 13,
 'F[As-](F)(F)(F)(F)F': 14,
 'O[P-](O)(O)(O)(O)O': 15,
 'O=[N+]([O-])[O-]': 16,
 'F[B-](F)F': 17,
 'F[P-](F)(F)(F)(F)F': 18,
 'F[B-](F)(F)F': 19,
 'O=S(=O)NS(=O)=O': 20,
 '[Na+]': 21,
 'N': 22,
 '[O-][Cl+3]([O-])([O-])[O-]': 23,
 '[Y+3]': 24,
 'O=S(=O)[N-]S(=O)(=O)F': 25,
 '[La+3]': 26,
 '[Cl-]': 27,
 '[n+]': 28,
 'N#C[N-]C#N': 29,
 'O=S(=O)[N-]S(=O)=O': 30,
 '[Cu+]': 31,
 'O=S(=O)(F)[N-]S(=O)(=O)F': 32,
 'O=C1O[B-](F)(F)OC1=O': 33,
 'F': 34,
 'N#C[S-]': 35,
 '[Ca+2]': 36,
 'O=CO[B-](O)(O)OC=O': 37,
 'C#N': 38,
 'n': 39,
 '[Pb+2]': 40,
 '[n-]': 41,
 '[B-]': 42,
 '[N+]': 43}

In [19]:
## Define the function for calculating number of fragments
## of the functional groups present in a query solvent
def calc_frag_fg_solvent(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fgs = identify_functional_groups(mol)
    counter_1 = 0; counter_2 = 0; counter_3 = 0; counter_4 = 0
    counter_5 = 0; counter_6 = 0; counter_7 = 0; counter_8 = 0
    counter_9 = 0; counter_10 = 0; counter_11 = 0; counter_12 = 0
    counter_13 = 0; counter_14 = 0; counter_15 = 0; counter_16 = 0
    counter_17 = 0; counter_18 = 0; counter_19 = 0; counter_20 = 0
    counter_21 = 0; counter_22 = 0; counter_23 = 0; counter_24 = 0
    counter_25 = 0; counter_26 = 0; counter_27 = 0; counter_28 = 0
    counter_29 = 0; counter_30 = 0; counter_31 = 0; counter_32 = 0
    # counter_33 = 0; counter_34 = 0; counter_35 = 0                  ## has 35 identifiers for single component electrolytes
    counter_33 = 0; counter_34 = 0; counter_35 = 0; counter_36 = 0
    counter_37 = 0; counter_38 = 0; counter_39 = 0; counter_40 = 0
    counter_41 = 0; counter_42 = 0
    for i in range(len(fgs)):
        if fgs[i].atoms == unique_functional_groups_solv[0]:
            counter_1 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[1]:
            counter_2 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[2]:
            counter_3 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[3]:
            counter_4 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[4]:
            counter_5 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[5]:
            counter_6 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[6]:
            counter_7 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[7]:
            counter_8 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[8]:
            counter_9 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[9]:
            counter_10 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[10]:
            counter_11 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[11]:
            counter_12 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[12]:
            counter_13 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[13]:
            counter_14 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[14]:
            counter_15 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[15]:
            counter_16 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[16]:
            counter_17 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[17]:
            counter_18 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[18]:
            counter_19 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[19]:
            counter_20 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[20]:
            counter_21 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[21]:
            counter_22 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[22]:
            counter_23 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[23]:
            counter_24 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[24]:
            counter_25 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[25]:
            counter_26 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[26]:
            counter_27 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[27]:
            counter_28 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[28]:
            counter_29 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[29]:
            counter_30 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[30]:
            counter_31 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[31]:
            counter_32 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[32]:
            counter_33 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[33]:
            counter_34 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[34]:
            counter_35 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[35]:
            counter_36 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[36]:
            counter_37 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[37]:
            counter_38 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[38]:
            counter_39 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[39]:
            counter_40 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[40]:
            counter_41 += 1
        elif fgs[i].atoms == unique_functional_groups_solv[41]:
            counter_42 += 1
    return counter_1, counter_2, counter_3, counter_4, \
        counter_5, counter_6, counter_7, counter_8, \
        counter_9, counter_10, counter_11, counter_12, \
        counter_13, counter_14, counter_15, counter_16, \
        counter_17, counter_18, counter_19, counter_20, \
        counter_21, counter_22, counter_23, counter_24, \
        counter_25, counter_26, counter_27, counter_28, \
        counter_29, counter_30, counter_31, counter_32, \
        counter_33, counter_34, counter_35, counter_36, \
        counter_37, counter_38, counter_39, counter_40, \
        counter_41, counter_42

In [20]:
# calc_frag_fg_solvent('C1(C(OC(=O)O1)F)F')

(0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0)

Functional group features for salt

In [21]:
## Define the function for calculating number of fragments
## of the functional groups present in a query salt
def calc_frag_fg_salt(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fgs = identify_functional_groups(mol)
    counter_1 = 0; counter_2 = 0; counter_3 = 0; counter_4 = 0
    counter_5 = 0; counter_6 = 0; counter_7 = 0; counter_8 = 0
    counter_9 = 0; counter_10 = 0; counter_11 = 0; counter_12 = 0
    counter_13 = 0; counter_14 = 0; counter_15 = 0; counter_16 = 0
    counter_17 = 0; counter_18 = 0; counter_19 = 0; counter_20 = 0
    counter_21 = 0; counter_22 = 0; counter_23 = 0; counter_24 = 0
    counter_25 = 0; counter_26 = 0; counter_27 = 0; counter_28 = 0
    counter_29 = 0; counter_30 = 0; counter_31 = 0; counter_32 = 0
    counter_33 = 0; counter_34 = 0; counter_35 = 0; counter_36 = 0
    counter_37 = 0; counter_38 = 0; counter_39 = 0; counter_40 = 0
    counter_41 = 0; counter_42 = 0; counter_43 = 0
    for i in range(len(fgs)):
        if fgs[i].atoms == unique_functional_groups_salt[0]:
            counter_1 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[1]:
            counter_2 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[2]:
            counter_3 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[3]:
            counter_4 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[4]:
            counter_5 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[5]:
            counter_6 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[6]:
            counter_7 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[7]:
            counter_8 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[8]:
            counter_9 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[9]:
            counter_10 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[10]:
            counter_11 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[11]:
            counter_12 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[12]:
            counter_13 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[13]:
            counter_14 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[14]:
            counter_15 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[15]:
            counter_16 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[16]:
            counter_17 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[17]:
            counter_18 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[18]:
            counter_19 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[19]:
            counter_20 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[20]:
            counter_21 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[21]:
            counter_22 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[22]:
            counter_23 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[23]:
            counter_24 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[24]:
            counter_25 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[25]:
            counter_26 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[26]:
            counter_27 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[27]:
            counter_28 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[28]:
            counter_29 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[29]:
            counter_30 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[30]:
            counter_31 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[31]:
            counter_32 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[32]:
            counter_33 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[33]:
            counter_34 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[34]:
            counter_35 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[35]:
            counter_36 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[36]:
            counter_37 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[37]:
            counter_38 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[38]:
            counter_39 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[39]:
            counter_40 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[40]:
            counter_41 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[41]:
            counter_42 += 1
        elif fgs[i].atoms == unique_functional_groups_salt[42]:
            counter_43 += 1
    return counter_1, counter_2, counter_3, counter_4, \
        counter_5, counter_6, counter_7, counter_8, \
        counter_9, counter_10, counter_11, counter_12, \
        counter_13, counter_14, counter_15, counter_16, \
        counter_17, counter_18, counter_19, counter_20, \
        counter_21, counter_22, counter_23, counter_24, \
        counter_25, counter_26, counter_27, counter_28, \
        counter_29, counter_30, counter_31, counter_32, \
        counter_33, counter_34, counter_35, counter_36, \
        counter_37, counter_38, counter_39, counter_40, \
        counter_41, counter_42, counter_43

In [58]:
def generate_features_solv(df, df_comb, df_add):
    prop_vec = []
    for i in range(len(df)):
        smiles_1 = df['solv_1_sm'][i]
        smiles_2 = df['solv_2_sm'][i]
        smiles_3 = df['solv_3_sm'][i]
        smiles_4 = df['solv_4_sm'][i]
        smiles_comb = df_comb['solv_comb_sm'][i]
        # print(smiles_comb)
        # mol = Chem.MolFromSmiles(smiles_comb)
        fr_func_grp_1, fr_func_grp_2, fr_func_grp_3, fr_func_grp_4,\
        fr_func_grp_5, fr_func_grp_6, fr_func_grp_7, fr_func_grp_8,\
        fr_func_grp_9, fr_func_grp_10, fr_func_grp_11, fr_func_grp_12,\
        fr_func_grp_13, fr_func_grp_14, fr_func_grp_15, fr_func_grp_16,\
        fr_func_grp_17, fr_func_grp_18, fr_func_grp_19, fr_func_grp_20,\
        fr_func_grp_21, fr_func_grp_22, fr_func_grp_23, fr_func_grp_24,\
        fr_func_grp_25, fr_func_grp_26, fr_func_grp_27, fr_func_grp_28,\
        fr_func_grp_29, fr_func_grp_30, fr_func_grp_31, fr_func_grp_32,\
        fr_func_grp_33, fr_func_grp_32, fr_func_grp_35, fr_func_grp_36,\
        fr_func_grp_37, fr_func_grp_38, fr_func_grp_39, fr_func_grp_40,\
        fr_func_grp_41, fr_func_grp_42  = calc_frag_fg_solvent(smiles_comb)
        mol_1 = Chem.MolFromSmiles(smiles_1)
        mol_comb = Chem.MolFromSmiles(smiles_comb)
        num_heavy_atoms = MolDescriptors.CalcNumHeavyAtoms(mol_comb)
        num_rot_bond_1 = MolDescriptors.CalcNumRotatableBonds(mol_1)
        solv_ratio_1 = df_add['solv_ratio_1'][i]; solv_ratio_2 = df_add['solv_ratio_2'][i]; solv_ratio_3 = df_add['solv_ratio_3'][i]; solv_ratio_4 = df_add['solv_ratio_4'][i]
        mol_wt_1 = MolDescriptors.CalcExactMolWt(mol_1)
        if df_add['solv_ratio_2'][i] == 0.0:
            num_rot_bond_2 = 0.0; num_rot_bond_3 = 0.0; num_rot_bond_4 = 0.0
            mol_wt_2 = 0.0; mol_wt_3 = 0.0; mol_wt_4 = 0.0

        elif (df_add['solv_ratio_2'][i] != 0.0 and df_add['solv_ratio_3'][i] == 0.0):
            # print(smiles_2)
            mol_2 = Chem.MolFromSmiles(smiles_2)
            num_rot_bond_2 = MolDescriptors.CalcNumRotatableBonds(mol_2)
            mol_wt_2 = MolDescriptors.CalcExactMolWt(mol_2)
            num_rot_bond_3 = 0.0; num_rot_bond_4 = 0.0
            mol_wt_3 = 0.0; mol_wt_4 = 0.0

        elif (df_add['solv_ratio_3'][i] != 0.0 and df_add['solv_ratio_4'][i] == 0.0):
            mol_2 = Chem.MolFromSmiles(smiles_2); mol_3 = Chem.MolFromSmiles(smiles_3)
            num_rot_bond_2 = MolDescriptors.CalcNumRotatableBonds(mol_2); num_rot_bond_3 = MolDescriptors.CalcNumRotatableBonds(mol_3)
            mol_wt_2 = MolDescriptors.CalcExactMolWt(mol_2); mol_wt_3 = MolDescriptors.CalcExactMolWt(mol_3)
            num_rot_bond_4 = 0.0; mol_wt_4 = 0.0

        else:
            mol_2 = Chem.MolFromSmiles(smiles_2); mol_3 = Chem.MolFromSmiles(smiles_3); mol_4 = Chem.MolFromSmiles(smiles_4)
            num_rot_bond_2 = MolDescriptors.CalcNumRotatableBonds(mol_2); num_rot_bond_3 = MolDescriptors.CalcNumRotatableBonds(mol_3); num_rot_bond_4 = MolDescriptors.CalcNumRotatableBonds(mol_4)
            mol_wt_2 = MolDescriptors.CalcExactMolWt(mol_2); mol_wt_3 = MolDescriptors.CalcExactMolWt(mol_3); mol_wt_4 = MolDescriptors.CalcExactMolWt(mol_4)

        vec = [smiles_comb] + [fr_func_grp_1] + [fr_func_grp_2] + [fr_func_grp_3] + [fr_func_grp_4] + \
        [fr_func_grp_5] + [fr_func_grp_6] + [fr_func_grp_7] + [fr_func_grp_8] + \
        [fr_func_grp_9] + [fr_func_grp_10] + [fr_func_grp_11] + [fr_func_grp_12] + \
        [fr_func_grp_13] + [fr_func_grp_14] + [fr_func_grp_15] + [fr_func_grp_16] + \
        [fr_func_grp_17] + [fr_func_grp_18] + [fr_func_grp_19] + [fr_func_grp_20] + \
        [fr_func_grp_21] + [fr_func_grp_22] + [fr_func_grp_23] + [fr_func_grp_24] + \
        [fr_func_grp_25] + [fr_func_grp_26] + [fr_func_grp_27] + [fr_func_grp_28] + \
        [fr_func_grp_29] + [fr_func_grp_30] + [fr_func_grp_31] + [fr_func_grp_32] + \
        [fr_func_grp_33] + [fr_func_grp_32] + [fr_func_grp_35] + [fr_func_grp_36] + \
        [fr_func_grp_37] + [fr_func_grp_38] + [fr_func_grp_39] + [fr_func_grp_40] + \
        [fr_func_grp_41] + [fr_func_grp_42] + [num_heavy_atoms] + [num_rot_bond_1] + \
        [num_rot_bond_2] + [num_rot_bond_3] + [num_rot_bond_4] + [solv_ratio_1] + \
        [solv_ratio_2] + [solv_ratio_3] + [solv_ratio_4] + [mol_wt_1] + [mol_wt_2] + \
        [mol_wt_3] + [mol_wt_4]
        prop_vec.append(vec)
    return prop_vec

In [28]:
solv_feature_names = ['solv_comb_sm', 'fr_func_grp_1_solv', 'fr_func_grp_2_solv', 'fr_func_grp_3_solv', 'fr_func_grp_4_solv',\
'fr_func_grp_5_solv', 'fr_func_grp_6_solv', 'fr_func_grp_7_solv', 'fr_func_grp_8_solv',\
'fr_func_grp_9_solv', 'fr_func_grp_10_solv', 'fr_func_grp_11_solv', 'fr_func_grp_12_solv',\
'fr_func_grp_13_solv', 'fr_func_grp_14_solv', 'fr_func_grp_15_solv', 'fr_func_grp_16_solv',\
'fr_func_grp_17_solv', 'fr_func_grp_18_solv', 'fr_func_grp_19_solv', 'fr_func_grp_20_solv',\
'fr_func_grp_21_solv', 'fr_func_grp_22_solv', 'fr_func_grp_23_solv', 'fr_func_grp_24_solv',\
'fr_func_grp_25_solv', 'fr_func_grp_26_solv', 'fr_func_grp_27_solv', 'fr_func_grp_28_solv',\
'fr_func_grp_29_solv', 'fr_func_grp_30_solv', 'fr_func_grp_31_solv', 'fr_func_grp_32_solv',\
'fr_func_grp_33_solv', 'fr_func_grp_34_solv', 'fr_func_grp_35_solv', 'fr_func_grp_36_solv',\
'fr_func_grp_37_solv', 'fr_func_grp_38_solv', 'fr_func_grp_39_solv', 'fr_func_grp_40_solv',\
'fr_func_grp_41_solv', 'fr_func_grp_42_solv', 'num_heavy_atom_solv', 'num_rot_bond_solv_1',\
'num_rot_bond_solv_2', 'num_rot_bond_solv_3', 'num_rot_bond_solv_4', 'solv_ratio_1', 'solv_ratio_2',\
'solv_ratio_3',	'solv_ratio_4',	'mol_wt_solv_1', 'mol_wt_solv_2', 'mol_wt_solv_3', 'mol_wt_solv_4']

In [37]:
df_comb.loc[df_comb['solv_comb_sm'] == 'O=C1OCCO1...']

,solv_comb_sm,salt_sm,conductivity_log


In [38]:
X_solv = generate_features_solv(df, df_comb, df_add)
len(X_solv[0])

56

In [39]:
len(solv_feature_names)

56

In [40]:
df_solv_feat = pd.DataFrame(X_solv)
df_solv_feat.columns = solv_feature_names
# df_new['temperature'] = df['temperature']
# df_new['conductivity_log'] = df['conductivity_log']
df_solv_feat

,solv_comb_sm,fr_func_grp_1_solv,fr_func_grp_2_solv,fr_func_grp_3_solv,fr_func_grp_4_solv,fr_func_grp_5_solv,fr_func_grp_6_solv,fr_func_grp_7_solv,fr_func_grp_8_solv,fr_func_grp_9_solv,...,num_rot_bond_solv_3,num_rot_bond_solv_4,solv_ratio_1,solv_ratio_2,solv_ratio_3,solv_ratio_4,mol_wt_solv_1,mol_wt_solv_2,mol_wt_solv_3,mol_wt_solv_4
0,O=C1OCCO1.COC(=O)OC,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.447059,0.552941,0.0,0.0,88.016044,90.031694,0.0,0.0
1,O=C1OCCO1.CCCCOC(=O)OC,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.422222,0.577778,0.0,0.0,88.016044,132.078644,0.0,0.0
2,O=C1OCCO1.CC(C)COC(=O)OC,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.422222,0.577778,0.0,0.0,88.016044,132.078644,0.0,0.0
3,O=C1OCCO1.CCC(C)OC(=O)OC,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.422222,0.577778,0.0,0.0,88.016044,132.078644,0.0,0.0
4,CCOC(=O)OCC,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.000000,0.000000,0.0,0.0,118.062994,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10191,CS(=O)C.COCCOC,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.500000,0.500000,0.0,0.0,78.013936,90.068080,0.0,0.0
10192,CS(=O)C.COCCOC,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.500000,0.500000,0.0,0.0,78.013936,90.068080,0.0,0.0
10193,CS(=O)C.COCCOC,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.500000,0.500000,0.0,0.0,78.013936,90.068080,0.0,0.0
10194,CS(=O)C.COCCOC,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.500000,0.500000,0.0,0.0,78.013936,90.068080,0.0,0.0


In [70]:
def generate_features_salt(df, df_add):
    prop_vec = []
    for i in range(len(df)):
        smiles = df['salt_sm'][i]
        # print(smiles)
        mol = Chem.MolFromSmiles(smiles)
        fr_func_grp_1, fr_func_grp_2, fr_func_grp_3, fr_func_grp_4,\
        fr_func_grp_5, fr_func_grp_6, fr_func_grp_7, fr_func_grp_8,\
        fr_func_grp_9, fr_func_grp_10, fr_func_grp_11, fr_func_grp_12,\
        fr_func_grp_13, fr_func_grp_14, fr_func_grp_15, fr_func_grp_16,\
        fr_func_grp_17, fr_func_grp_18, fr_func_grp_19, fr_func_grp_20,\
        fr_func_grp_21, fr_func_grp_22, fr_func_grp_23, fr_func_grp_24,\
        fr_func_grp_25, fr_func_grp_26, fr_func_grp_27, fr_func_grp_28,\
        fr_func_grp_29, fr_func_grp_30, fr_func_grp_31, fr_func_grp_32,\
        fr_func_grp_33, fr_func_grp_32, fr_func_grp_35, fr_func_grp_36, \
        fr_func_grp_37, fr_func_grp_38, fr_func_grp_39, fr_func_grp_40, \
        fr_func_grp_41, fr_func_grp_42, fr_func_grp_43 = calc_frag_fg_salt(smiles)
        num_heavy_atoms = MolDescriptors.CalcNumHeavyAtoms(mol)
        num_rot_bond = MolDescriptors.CalcNumRotatableBonds(mol)
        mol_wt = MolDescriptors.CalcExactMolWt(mol)
        conc_salt = df_add['conc_salt'][i]
        temp = df_add['temperature'][i]

        vec = [smiles] + [fr_func_grp_1] + [fr_func_grp_2] + [fr_func_grp_3] + [fr_func_grp_4] + \
        [fr_func_grp_5] + [fr_func_grp_6] + [fr_func_grp_7] + [fr_func_grp_8] + \
        [fr_func_grp_9] + [fr_func_grp_10] + [fr_func_grp_11] + [fr_func_grp_12] + \
        [fr_func_grp_13] + [fr_func_grp_14] + [fr_func_grp_15] + [fr_func_grp_16] + \
        [fr_func_grp_17] + [fr_func_grp_18] + [fr_func_grp_19] + [fr_func_grp_20] + \
        [fr_func_grp_21] + [fr_func_grp_22] + [fr_func_grp_23] + [fr_func_grp_24] + \
        [fr_func_grp_25] + [fr_func_grp_26] + [fr_func_grp_27] + [fr_func_grp_28] + \
        [fr_func_grp_29] + [fr_func_grp_30] + [fr_func_grp_31] + [fr_func_grp_32] + \
        [fr_func_grp_33] + [fr_func_grp_32] + [fr_func_grp_35] + [fr_func_grp_36] + \
        [fr_func_grp_37] + [fr_func_grp_38] + [fr_func_grp_39] + [fr_func_grp_40] + \
        [fr_func_grp_41] + [fr_func_grp_42] + [fr_func_grp_43] + [num_heavy_atoms] + \
        [num_rot_bond] + [mol_wt] + [conc_salt] + [temp]
        # [fr_func_grp_41] + [fr_func_grp_42] + [fr_func_grp_43] + descriptors_3d
        prop_vec.append(vec)
    return prop_vec

In [42]:
salt_feature_names = ['salt_smiles', 'fr_func_grp_1_salt', 'fr_func_grp_2_salt', 'fr_func_grp_3_salt', 'fr_func_grp_4_salt',\
'fr_func_grp_5_salt', 'fr_func_grp_6_salt', 'fr_func_grp_7_salt', 'fr_func_grp_8_salt',\
'fr_func_grp_9_salt', 'fr_func_grp_10_salt', 'fr_func_grp_11_salt', 'fr_func_grp_12_salt',\
'fr_func_grp_13_salt', 'fr_func_grp_14_salt', 'fr_func_grp_15_salt', 'fr_func_grp_16_salt',\
'fr_func_grp_17_salt', 'fr_func_grp_18_salt', 'fr_func_grp_19_salt', 'fr_func_grp_20_salt',\
'fr_func_grp_21_salt', 'fr_func_grp_22_salt', 'fr_func_grp_23_salt', 'fr_func_grp_24_salt',\
'fr_func_grp_25_salt', 'fr_func_grp_26_salt', 'fr_func_grp_27_salt', 'fr_func_grp_28_salt',\
'fr_func_grp_29_salt', 'fr_func_grp_30_salt', 'fr_func_grp_31_salt', 'fr_func_grp_32_salt',\
'fr_func_grp_33_salt', 'fr_func_grp_34_salt', 'fr_func_grp_35_salt', 'fr_func_grp_36_salt',\
'fr_func_grp_37_salt', 'fr_func_grp_38_salt', 'fr_func_grp_39_salt', 'fr_func_grp_40_salt',\
'fr_func_grp_41_salt', 'fr_func_grp_42_salt', 'fr_func_grp_43_salt', 'num_heavy_atom_salt',\
'num_rot_bond_salt', 'mw_salt', 'conc_salt', 'temperature']

In [44]:
X_salt = generate_features_salt(df)
len(X_salt[0])

49

In [45]:
df_salt_feat = pd.DataFrame(X_salt)
df_salt_feat.columns = salt_feature_names
df_salt_feat

,salt_smiles,fr_func_grp_1_salt,fr_func_grp_2_salt,fr_func_grp_3_salt,fr_func_grp_4_salt,fr_func_grp_5_salt,fr_func_grp_6_salt,fr_func_grp_7_salt,fr_func_grp_8_salt,fr_func_grp_9_salt,...,fr_func_grp_39_salt,fr_func_grp_40_salt,fr_func_grp_41_salt,fr_func_grp_42_salt,fr_func_grp_43_salt,num_heavy_atom_salt,num_rot_bond_salt,mw_salt,conc_salt,temperature
0,[Li+].F[P-](F)(F)(F)(F)F,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,8,0,151.980186,1.0,25.0
1,[Li+].F[P-](F)(F)(F)(F)F,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,8,0,151.980186,1.0,25.0
2,[Li+].F[P-](F)(F)(F)(F)F,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,8,0,151.980186,1.0,25.0
3,[Li+].F[P-](F)(F)(F)(F)F,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,8,0,151.980186,1.0,25.0
4,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,16,2,286.933298,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10191,[Li+].F[P-](F)(F)(F)(F)F,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,8,0,151.980186,0.1,30.0
10192,[Li+].F[P-](F)(F)(F)(F)F,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,8,0,151.980186,0.5,30.0
10193,[Li+].F[P-](F)(F)(F)(F)F,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,8,0,151.980186,1.0,30.0
10194,[Li+].F[P-](F)(F)(F)(F)F,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,8,0,151.980186,1.5,30.0


In [46]:
all_feature_names = solv_feature_names + salt_feature_names
df_all = pd.concat([df_solv_feat, df_salt_feat], axis=1, ignore_index=True)
df_all.columns = all_feature_names

df_all['conductivity_log'] = df['conductivity_log']
df_all

,solv_comb_sm,fr_func_grp_1_solv,fr_func_grp_2_solv,fr_func_grp_3_solv,fr_func_grp_4_solv,fr_func_grp_5_solv,fr_func_grp_6_solv,fr_func_grp_7_solv,fr_func_grp_8_solv,fr_func_grp_9_solv,...,fr_func_grp_40_salt,fr_func_grp_41_salt,fr_func_grp_42_salt,fr_func_grp_43_salt,num_heavy_atom_salt,num_rot_bond_salt,mw_salt,conc_salt,temperature,conductivity_log
0,O=C1OCCO1.COC(=O)OC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,25.0,1.068186
1,O=C1OCCO1.CCCCOC(=O)OC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,25.0,0.770852
2,O=C1OCCO1.CC(C)COC(=O)OC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,25.0,0.732394
3,O=C1OCCO1.CCC(C)OC(=O)OC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,25.0,0.812913
4,CCOC(=O)OCC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,16,2,286.933298,1.0,10.0,0.322219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10191,CS(=O)C.COCCOC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,0.1,30.0,0.672098
10192,CS(=O)C.COCCOC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,0.5,30.0,1.139879
10193,CS(=O)C.COCCOC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,30.0,1.204120
10194,CS(=O)C.COCCOC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.5,30.0,1.167317


In [47]:
reorder_feature_names = ['solv_comb_sm', 'salt_smiles', 'fr_func_grp_1_solv', 'fr_func_grp_2_solv', 'fr_func_grp_3_solv', 'fr_func_grp_4_solv',\
'fr_func_grp_5_solv', 'fr_func_grp_6_solv', 'fr_func_grp_7_solv', 'fr_func_grp_8_solv',\
'fr_func_grp_9_solv', 'fr_func_grp_10_solv', 'fr_func_grp_11_solv', 'fr_func_grp_12_solv',\
'fr_func_grp_13_solv', 'fr_func_grp_14_solv', 'fr_func_grp_15_solv', 'fr_func_grp_16_solv',\
'fr_func_grp_17_solv', 'fr_func_grp_18_solv', 'fr_func_grp_19_solv', 'fr_func_grp_20_solv',\
'fr_func_grp_21_solv', 'fr_func_grp_22_solv', 'fr_func_grp_23_solv', 'fr_func_grp_24_solv',\
'fr_func_grp_25_solv', 'fr_func_grp_26_solv', 'fr_func_grp_27_solv', 'fr_func_grp_28_solv',\
'fr_func_grp_29_solv', 'fr_func_grp_30_solv', 'fr_func_grp_31_solv', 'fr_func_grp_32_solv',\
'fr_func_grp_33_solv', 'fr_func_grp_34_solv', 'fr_func_grp_35_solv', 'fr_func_grp_36_solv',\
'fr_func_grp_37_solv', 'fr_func_grp_38_solv', 'fr_func_grp_39_solv', 'fr_func_grp_40_solv',\
'fr_func_grp_41_solv', 'fr_func_grp_42_solv']
reorder_feature_names += ['num_heavy_atom_solv', 'num_rot_bond_solv_1', 'num_rot_bond_solv_2',\
                          'num_rot_bond_solv_3', 'num_rot_bond_solv_4', 'solv_ratio_1', 'solv_ratio_2',\
                          'solv_ratio_3',	'solv_ratio_4',	'mol_wt_solv_1', 'mol_wt_solv_2', 'mol_wt_solv_3', 'mol_wt_solv_4']
reorder_feature_names += ['fr_func_grp_1_salt', 'fr_func_grp_2_salt', 'fr_func_grp_3_salt', 'fr_func_grp_4_salt',\
'fr_func_grp_5_salt', 'fr_func_grp_6_salt', 'fr_func_grp_7_salt', 'fr_func_grp_8_salt',\
'fr_func_grp_9_salt', 'fr_func_grp_10_salt', 'fr_func_grp_11_salt', 'fr_func_grp_12_salt',\
'fr_func_grp_13_salt', 'fr_func_grp_14_salt', 'fr_func_grp_15_salt', 'fr_func_grp_16_salt',\
'fr_func_grp_17_salt', 'fr_func_grp_18_salt', 'fr_func_grp_19_salt', 'fr_func_grp_20_salt',\
'fr_func_grp_21_salt', 'fr_func_grp_22_salt', 'fr_func_grp_23_salt', 'fr_func_grp_24_salt',\
'fr_func_grp_25_salt', 'fr_func_grp_26_salt', 'fr_func_grp_27_salt', 'fr_func_grp_28_salt',\
'fr_func_grp_29_salt', 'fr_func_grp_30_salt', 'fr_func_grp_31_salt', 'fr_func_grp_32_salt',\
'fr_func_grp_33_salt', 'fr_func_grp_34_salt', 'fr_func_grp_35_salt', 'fr_func_grp_36_salt',\
'fr_func_grp_37_salt', 'fr_func_grp_38_salt', 'fr_func_grp_39_salt', 'fr_func_grp_40_salt',\
'fr_func_grp_41_salt', 'fr_func_grp_42_salt', 'fr_func_grp_43_salt']
reorder_feature_names += ['num_heavy_atom_salt','num_rot_bond_salt', 'mw_salt', 'conc_salt', 'temperature']
reorder_feature_names += ['conductivity_log']
len(reorder_feature_names)

106

In [48]:
df_all.reset_index(inplace=True)
df_all

,index,solv_comb_sm,fr_func_grp_1_solv,fr_func_grp_2_solv,fr_func_grp_3_solv,fr_func_grp_4_solv,fr_func_grp_5_solv,fr_func_grp_6_solv,fr_func_grp_7_solv,fr_func_grp_8_solv,...,fr_func_grp_40_salt,fr_func_grp_41_salt,fr_func_grp_42_salt,fr_func_grp_43_salt,num_heavy_atom_salt,num_rot_bond_salt,mw_salt,conc_salt,temperature,conductivity_log
0,0,O=C1OCCO1.COC(=O)OC,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,25.0,1.068186
1,1,O=C1OCCO1.CCCCOC(=O)OC,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,25.0,0.770852
2,2,O=C1OCCO1.CC(C)COC(=O)OC,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,25.0,0.732394
3,3,O=C1OCCO1.CCC(C)OC(=O)OC,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,25.0,0.812913
4,4,CCOC(=O)OCC,0,0,0,0,0,0,0,0,...,0,0,0,0,16,2,286.933298,1.0,10.0,0.322219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10191,10191,CS(=O)C.COCCOC,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,0.1,30.0,0.672098
10192,10192,CS(=O)C.COCCOC,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,0.5,30.0,1.139879
10193,10193,CS(=O)C.COCCOC,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,30.0,1.204120
10194,10194,CS(=O)C.COCCOC,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.5,30.0,1.167317


In [49]:
## Reorder the columns of the dataframe
df_all_new = df_all.reindex(columns=reorder_feature_names)
df_all_new

,solv_comb_sm,salt_smiles,fr_func_grp_1_solv,fr_func_grp_2_solv,fr_func_grp_3_solv,fr_func_grp_4_solv,fr_func_grp_5_solv,fr_func_grp_6_solv,fr_func_grp_7_solv,fr_func_grp_8_solv,...,fr_func_grp_40_salt,fr_func_grp_41_salt,fr_func_grp_42_salt,fr_func_grp_43_salt,num_heavy_atom_salt,num_rot_bond_salt,mw_salt,conc_salt,temperature,conductivity_log
0,O=C1OCCO1.COC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,25.0,1.068186
1,O=C1OCCO1.CCCCOC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,25.0,0.770852
2,O=C1OCCO1.CC(C)COC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,25.0,0.732394
3,O=C1OCCO1.CCC(C)OC(=O)OC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,25.0,0.812913
4,CCOC(=O)OCC,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,16,2,286.933298,1.0,10.0,0.322219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10191,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,0.1,30.0,0.672098
10192,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,0.5,30.0,1.139879
10193,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.0,30.0,1.204120
10194,CS(=O)C.COCCOC,[Li+].F[P-](F)(F)(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,8,0,151.980186,1.5,30.0,1.167317


In [50]:
df_all_new.to_csv('all_qsar_fgrp_cond.csv', index=False)

**Generate for OOD**

In [71]:
df_ood = pd.read_csv('cond_ood_final_comp.csv')
df_ood

,solv_comb_sm,salt_sm,true_conductivity_log,class,group
0,C(COCCOCCOCC(F)(F)F)(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.333390,Ether,in-house
1,C(COCCOCCOCC(F)(F)F)(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.297290,Ether,in-house
2,C(COCCOCCOCC(F)(F)F)(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.254010,Ether,in-house
3,C(COCCOCCOCC(F)(F)F)(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.202220,Ether,in-house
4,C(COCCOCCOCC(F)(F)F)(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.137990,Ether,in-house
...,...,...,...,...,...
166,CS(=O)(=O)OCC(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.503791,Others,literature
167,CS(=O)(=O)OCC(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.474216,Others,literature
168,CO[Si](C)(CCC(F)(F)F)OC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.585027,Others,literature
169,CO[Si](C)(CCC(F)(F)F)OC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.356547,Others,literature


In [72]:
# df_ood_comb = df_ood.copy()
df_ood_add = pd.read_csv('cond_ood_final_add.csv')
df_ood_add

,solv_ratio_1,solv_ratio_2,solv_ratio_3,solv_ratio_4,mol_wt_solv_1,mol_wt_solv_2,mol_wt_solv_3,mol_wt_solv_4,mol_wt_salt,conc_salt,temperature
0,1,0,0,0,270.069064,0,0,0,186.939685,1.0,80
1,1,0,0,0,270.069064,0,0,0,186.939685,1.0,70
2,1,0,0,0,270.069064,0,0,0,186.939685,1.0,60
3,1,0,0,0,270.069064,0,0,0,186.939685,1.0,50
4,1,0,0,0,270.069064,0,0,0,186.939685,1.0,40
...,...,...,...,...,...,...,...,...,...,...,...
166,1,0,0,0,177.991150,0,0,0,186.939685,1.2,25
167,1,0,0,0,177.991150,0,0,0,186.939685,1.0,25
168,1,0,0,0,202.063691,0,0,0,186.939685,1.0,25
169,1,0,0,0,202.063691,0,0,0,186.939685,3.0,25


In [73]:
df_ood_comb = df_ood.copy()
df_ood['solv_1_sm'] = df_ood['solv_comb_sm']
df_ood['solv_2_sm'] = np.nan
df_ood['solv_3_sm'] = np.nan
df_ood['solv_4_sm'] = np.nan
# df_ood_comb['salt_sm'] = df_ood['salt_sm']
df_ood['conductivity_log'] = df_ood['true_conductivity_log']

In [74]:
X_solv_ood = generate_features_solv(df_ood, df_ood_comb, df_ood_add)
len(X_solv_ood[0])

56

In [76]:
X_salt_ood = generate_features_salt(df_ood, df_ood_add)
len(X_salt_ood[0])

49

In [77]:
df_solv_ood = pd.DataFrame(X_solv_ood)
df_solv_ood.columns = solv_feature_names
df_salt_ood = pd.DataFrame(X_salt_ood)
df_salt_ood.columns = salt_feature_names

In [78]:
df_ood_all = pd.concat([df_solv_ood, df_salt_ood], axis=1, ignore_index=True)
df_ood_all.columns = all_feature_names

df_ood_all['conductivity_log'] = df_ood['conductivity_log']
df_ood_all

,solv_comb_sm,fr_func_grp_1_solv,fr_func_grp_2_solv,fr_func_grp_3_solv,fr_func_grp_4_solv,fr_func_grp_5_solv,fr_func_grp_6_solv,fr_func_grp_7_solv,fr_func_grp_8_solv,fr_func_grp_9_solv,...,fr_func_grp_40_salt,fr_func_grp_41_salt,fr_func_grp_42_salt,fr_func_grp_43_salt,num_heavy_atom_salt,num_rot_bond_salt,mw_salt,conc_salt,temperature,conductivity_log
0,C(COCCOCCOCC(F)(F)F)(F)(F)F,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,80,0.333390
1,C(COCCOCCOCC(F)(F)F)(F)(F)F,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,70,0.297290
2,C(COCCOCCOCC(F)(F)F)(F)(F)F,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,60,0.254010
3,C(COCCOCCOCC(F)(F)F)(F)(F)F,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,50,0.202220
4,C(COCCOCCOCC(F)(F)F)(F)(F)F,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,40,0.137990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,CS(=O)(=O)OCC(F)(F)F,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.2,25,0.503791
167,CS(=O)(=O)OCC(F)(F)F,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,25,0.474216
168,CO[Si](C)(CCC(F)(F)F)OC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,25,-0.585027
169,CO[Si](C)(CCC(F)(F)F)OC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,3.0,25,-0.356547


In [79]:
df_ood_all.reset_index(inplace=True)
df_ood_all_new = df_ood_all.reindex(columns=reorder_feature_names)
df_ood_all_new

,solv_comb_sm,salt_smiles,fr_func_grp_1_solv,fr_func_grp_2_solv,fr_func_grp_3_solv,fr_func_grp_4_solv,fr_func_grp_5_solv,fr_func_grp_6_solv,fr_func_grp_7_solv,fr_func_grp_8_solv,...,fr_func_grp_40_salt,fr_func_grp_41_salt,fr_func_grp_42_salt,fr_func_grp_43_salt,num_heavy_atom_salt,num_rot_bond_salt,mw_salt,conc_salt,temperature,conductivity_log
0,C(COCCOCCOCC(F)(F)F)(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,80,0.333390
1,C(COCCOCCOCC(F)(F)F)(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,70,0.297290
2,C(COCCOCCOCC(F)(F)F)(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,60,0.254010
3,C(COCCOCCOCC(F)(F)F)(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,50,0.202220
4,C(COCCOCCOCC(F)(F)F)(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,40,0.137990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,CS(=O)(=O)OCC(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.2,25,0.503791
167,CS(=O)(=O)OCC(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,25,0.474216
168,CO[Si](C)(CCC(F)(F)F)OC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,1.0,25,-0.585027
169,CO[Si](C)(CCC(F)(F)F)OC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0,0,0,0,0,0,0,0,...,0,0,0,0,10,2,186.939685,3.0,25,-0.356547


In [80]:
df_ood_all_new.to_csv('ood_qsar_fgrp_cond.csv', index=False)